In [1]:
import numpy as np

np.random.seed(123)
X = np.load('../generated_games/features-40k.npy')
Y = np.load('../generated_games/labels-40k.npy')
print("features", X.shape)
print("labels", Y.shape)

samples = X.shape[0]
size = 9
input_shape = (size, size, 1)

X = X.reshape(samples, size, size, 1)

train_samples = int(0.9 * samples)
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

features (41439, 1, 9, 9)
labels (41439, 81)
X_train (37295, 9, 9, 1)
Y_train (37295, 81)
X_test (4144, 9, 9, 1)
Y_test (4144, 81)


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv2D(filters=48,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Conv2D(48, (3, 3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))
model.add(Flatten()) 
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(size * size, activation='softmax')) 
model.summary()

2024-11-24 00:30:04.142626: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-11-24 00:30:04.142655: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-24 00:30:04.142660: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-24 00:30:04.142923: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-24 00:30:04.142937: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 9, 9, 48)       │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 9, 9, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 9, 9, 48)       │        20,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 81)             │        41,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,545 (1.74 MB)

 Trainable params: 456,545 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
import time

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Record the start time
start_time = time.time()
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=200,
          verbose=1,
          validation_data=(X_test, Y_test))
# Record the end time
end_time = time.time()
# Calculate and display the wall time
print(f"Wall time: {end_time - start_time:.6f} seconds")

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/200


2024-11-24 00:30:04.654325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


583/583 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.0191 - loss: 4.3911 - val_accuracy: 0.0227 - val_loss: 4.3171
Epoch 2/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0223 - loss: 4.3213 - val_accuracy: 0.0275 - val_loss: 4.2727
Epoch 3/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0212 - loss: 4.2921 - val_accuracy: 0.0251 - val_loss: 4.2544
Epoch 4/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0232 - loss: 4.2706 - val_accuracy: 0.0253 - val_loss: 4.2390
Epoch 5/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0244 - loss: 4.2529 - val_accuracy: 0.0241 - val_loss: 4.2200
Epoch 6/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0246 - loss: 4.2308 - val_accuracy: 0.0227 - val_loss: 4.2009
Epoch 7/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.0260 - loss: 4.1986 - val_accuracy: 0.0256 - val_loss: 4.1761
Epoch 8/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0254 - loss: 4.1792 - val_accurac

In [5]:
test_board = np.array([[
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 0,  1, -1,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
]]).reshape(1, 9, 9, 1)
move_probs = model.predict(test_board)[0]
i = 0
for row in range(9):
    row_formatted = []
    for col in range(9):
        row_formatted.append('{:.3f}'.format(move_probs[i]))
        i += 1
    print(' '.join(row_formatted))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
0.000 0.001 0.001 0.001 0.001 0.001 0.000 0.001 0.000
0.001 0.009 0.020 0.042 0.021 0.010 0.005 0.003 0.001
0.002 0.024 0.006 0.117 0.034 0.030 0.001 0.004 0.001
0.002 0.018 0.024 0.017 0.012 0.011 0.005 0.004 0.001
0.004 0.015 0.007 0.003 0.014 0.018 0.014 0.007 0.002
0.002 0.023 0.029 0.028 0.096 0.039 0.014 0.006 0.001
0.001 0.016 0.004 0.068 0.048 0.027 0.001 0.005 0.001
0.000 0.006 0.011 0.022 0.013 0.008 0.008 0.003 0.001
0.000 0.000 0.001 0.001 0.001 0.001 0.000 0.000 0.000


Starlight MacBook Air M2:
> Wall time: 1366.246495 seconds

> Test loss: 3.5951743125915527
> Test accuracy: 0.13778957724571228

MacBookPro M1 Pro:
> Wall time: 1680.059903 seconds

> Test loss: 3.5705721378326416
> Test accuracy: 0.1546814739704132